In [ ]:
import pandas as pd
import humanize

In [ ]:
input_path = '../../tmp-data/kline_rb_signals_merged_df'

In [ ]:
df = pd.read_csv(f'{input_path}.csv', parse_dates=['timestamp'], index_col='timestamp')
df

In [ ]:
import random
from backtesting import Strategy
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

class RangeBarStrategyPreCalc(Strategy):

    per_trade_risk = 0.02
    trades_buy_open_at_time_of_buy = []
    trades_sell_open_at_time_of_sell = []
    trades_open_per_tick = []
    
    
    def init(self):
       self.false_buys = 0
       self.false_sells = 0
       self.trades_buy_open_at_time_of_buy = []
       self.trades_sell_open_at_time_of_sell = []
       self.trades_open_per_tick = []

    # Stop Loss Price = Liquidation Price - (Liquidation Price - Entry Price) / Leverage * Stop Loss Ratio
    def next(self):
        ts = self.data.index[-1]
        current_close = self.data.Close[-1]
        range_size = self.data.average_adr[-1] * 0.1
        potential_profit = self.data.average_adr[-1] * 0.15
        
        pre_calc_signal = self.data.signal[-1]
     
        coin_toss = random.choice([True, False])    

        order_placed = False
        if pre_calc_signal > 0:
            order_placed = True        
            self.buy(size=self.per_trade_risk * abs(pre_calc_signal), sl=current_close - range_size, tp=current_close + potential_profit)
        elif pre_calc_signal < 0:
            order_placed = True    
            self.sell(size=self.per_trade_risk * abs(pre_calc_signal), sl=current_close + range_size , tp=current_close - potential_profit)

        # order counts    
        if order_placed:    
          is_short_count = len(list(filter(lambda t: t.is_short, self.trades)))
          is_long_count = len(list(filter(lambda t: t.is_long, self.trades)))
          self.trades_buy_open_at_time_of_buy.append(is_long_count)
          self.trades_sell_open_at_time_of_sell.append(is_short_count)
        is_short_count = len(list(filter(lambda t: t.is_short, self.trades)))
        is_long_count = len(list(filter(lambda t: t.is_long, self.trades)))
        self.trades_open_per_tick.append({'timestamp': ts, 'buy': is_long_count, 'sell': is_short_count})

In [ ]:
from backtesting import Backtest
bt = Backtest(df, RangeBarStrategyPreCalc, hedging=False, cash=100000, margin=1/20, commission=(0.04 / 100), exclusive_orders=False)
stats = bt.run()
stats